In [9]:
import nltk
from nltk import word_tokenize
import dill
import numpy as np
from multiprocessing import Pool, Lock, Value
from IPython.display import clear_output
from pymystem3 import Mystem
import re
import inscriptis
import os
from bs4 import BeautifulSoup
import Stemmer
import requests
import subprocess
from bs4.element import Comment
import time

import bokeh
import bokeh.models
import bokeh.plotting
import numpy as np
import os
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tf_sentencepiece  # Not used directly but needed to import TF ops.
import sklearn.metrics.pairwise
import tqdm
from nltk.corpus import stopwords

DOC_NUM = 38114
DATA = 'content/'
DICT_FOLDER = 'dicts/'
EXTRACTED = 'extracted/'
EXTRACTED_RAW = 'extracted_raw/'
EXTRACTED_PROCESSED = 'extracted_processed/'

In [21]:
stopwords_rus = set(stopwords.words('russian'))
stopwords_eng = set(stopwords.words('english'))

***Загрузка***

In [13]:
def load_url_to_num():
    url_to_num = dict()
    num_to_url = dict()
    with open('urls.numerate.txt', 'r') as f:
        for line in f:
            #print(line)
            line = line.strip('\n')
            line = line.split('\t')
            url_to_num[line[1]] = line[0]
            num_to_url[line[0]] = line[1]
    return url_to_num, num_to_url

In [14]:
def load_num_to_path():
    num_to_path = dict()
    url_to_num , num_to_url = load_url_to_num()
    counter = 0
    for i in os.listdir(DATA):
        for j in os.listdir(DATA + i):
            path = DATA + i + '/' + j
            with open(path, 'r', errors='replace') as f:
                url = f.readline().strip()
            num = url_to_num[url]
            num_to_path[num] = path
            counter += 1
            if counter % 100 == 0:
                clear_output(True)
                print(counter)
    return num_to_path, num_to_url

In [9]:
num_to_path, num_to_url = load_num_to_path()

38100


In [15]:
with open('num_to_path.dat', 'wb') as f:
    dill.dump(num_to_path, f)
with open('num_to_url.dat', 'wb') as f:
    dill.dump(num_to_url, f)

In [ ]:
with open('num_to_path.dat', 'rb') as f:
    num_to_path = dill.load(f)
with open('num_to_url.dat', 'rb') as f:
    num_to_url = dill.load(f)

***Добавление документов*** <br>
Часть документов испорчена. Я скачал их из архива за 2017 год и добавил.<br>
Документы лежат в папке manually_downloaded и пронуменрованы doc_id.

In [16]:
corrupted_nums = []
for i in range(1, DOC_NUM + 1):
    path = num_to_path[str(i)]
    with open(path, 'r', errors='replace') as f:
        doc = list(f)
    body = ' '.join(doc[1:]).strip()
    if body.lower() == 'aAqQsSwWdDeEfFrR'.lower():
        corrupted_nums.append(i)
    if i % 100 == 0:
        clear_output(True)
        print(i)

38100


In [ ]:
for num in corrupted_nums:
    url = num_to_url[str(num)]
    path = num_to_path[str(num)]
    try:
        with open('manually_downloaded/' + str(num) + '.html', 'r', errors='replace') as f:
            html_doc = f.read()
        doc = '\n'.join([url, html_doc])
        with open(path, 'w') as f:
            f.write(doc)
    except BaseException:
        print(url)

***Извлечение текста из файлов, обработка текста, лемматизация.***

In [18]:
def process_0(text):
    text = text.replace('-\n', "")
    #text = re.sub('(?: -|- |[^\w-]+)', ' ', text) #удаляем все знаки препинания и т.п., но хотим оставить
                                                  #слова вроде петропавловск-камчатский
    text = re.sub("\W+", r' ', text)
    text = ' '.join(re.split("(\d+)", text))
    return text   
def process_1(word):
    word = word.replace(u'ё', u'е')
    word = word.replace('_', '')
    word = word.replace('_', '')
    word = re.sub(' +', ' ', word)
    word = word.strip()
    return word

In [38]:
def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

In [39]:
def bs4_text_from_html(body):
    soup = BeautifulSoup(body, 'html.parser')
    if not bool(soup.find()):
        raise BaseException
    
    texts = soup.findAll(text=True)
    visible_texts = filter(tag_visible, texts)
    return u" ".join(t.strip() for t in visible_texts)

In [40]:
def extract_text(num):
    try:
        stemmer_rus = Mystem()
        stemmer_eng = Stemmer.Stemmer('english')
        path = num_to_path[str(num)]
        cur_doc = dict()
        with open(path, 'r', errors='replace') as f:
            doc = list(f)
        url = num_to_url[str(num)]
        doc = ''.join(doc[1:])
        bs = BeautifulSoup(doc)
        title = bs.find('title')
        if title == None:
            cur_doc['title'] = ''
        else:
            cur_doc['title'] = title.text 
        try:
            text = bs4_text_from_html(doc)
        except BaseException:
            text = inscriptis.get_text(doc)
        cur_doc['text'] = text
        cur_doc['title'] = process_0(cur_doc['title']).lower()
        cur_doc['title'] = ''.join(stemmer_rus.lemmatize(cur_doc['title'])).strip()
        cur_doc['title'] = re.sub("\W+", r' ', cur_doc['title'])
        cur_doc['title'] = ' ' .join(stemmer_eng.stemWords(cur_doc['title'].split(' ')))
        cur_doc['title'] = ' '.join(list(map(process_1, nltk.tokenize.word_tokenize(cur_doc['title']))))
        cur_doc['text'] = process_0(cur_doc['text']).lower()
        cur_doc['text'] = ''.join(stemmer_rus.lemmatize(cur_doc['text'])).strip()
        cur_doc['text'] = re.sub("\W+", r' ', cur_doc['text'])
        cur_doc['text'] = ' '.join(stemmer_eng.stemWords(cur_doc['text'].split(' ')))
        cur_doc['text'] = ' '.join(list(map(process_1, nltk.tokenize.word_tokenize(cur_doc['text']))))
        with open(EXTRACTED + str(num) + '.dat', 'wb') as f:
            dill.dump(cur_doc, f)
    except BaseException:
        with open('exceptions/' + str(num), 'w') as f:
            f.write('failed')

In [41]:
def extract_raw_text(num):
    try:
        path = num_to_path[str(num)]
        with open(path, 'r', errors='replace') as f:
            doc = list(f)
        doc = ''.join(doc[1:])
        bs = BeautifulSoup(doc)
        title = bs.find('title')
        if title == None:
            title = ''
        else:
            title = title.text
        try:
            text = bs4_text_from_html(doc)
        except BaseException:
            text = inscriptis.get_text(doc)
        final_text = ' '.join([title, text])
        #final_text = re.sub('\n', ' ', final_text)
        final_text = final_text.replace('-', ' ')
        final_text = re.sub(' +', ' ', final_text)
        with open(EXTRACTED_RAW + str(num) + '.dat', 'w', errors='replace') as f:
            f.write(final_text)
        final_text = process_0(final_text)
        with open(EXTRACTED_PROCESSED + str(num) + '.dat', 'w', errors='replace') as f:
            f.write(final_text)
    except BaseException:
        with open('exceptions/' + str(num) + '.dat', 'w') as f:
            f.write('failed')

In [42]:
def wrapper_extract_raw(doc_id):
    extract_raw_text(doc_id)
    with mutex:
        val.value += 1
        if val.value % 100 == 0:
            clear_output(True)
            print(val.value, flush=True)
    return None

In [43]:
def wrapper_extract(doc_id):
    extract_text(doc_id)
    with mutex:
        val.value += 1
        if val.value % 100 == 0:
            clear_output(True)
            print(val.value, flush=True)
    return None

***Сбор статистики документов***

In [44]:
class CorpusStatistics(dict):
    def __init__(self):
        self['df'] = dict()
        self['cf'] = dict()
        self['count docs'] = 0
        self['count lemms'] = 0
        self['avgdl'] = 0
    def idf(self, word):
        df = 0
        if word in self['df']:
            df = self['df'][word]
        if df == 0:
            return 0
        return self['count docs'] / df
    def log_idf(self, word):
        idf = self.idf(word)
        if idf == 0:
            return 0
        return np.log(idf)
    def icf(self, word):
        if word in self['cf']:
            cf = self['cf'][word]
        else:
            cf = 0
        if cf == 0:
            return 0
        return self['count lemms'] / cf
    def log_icf(self, word):
        icf = self.icf(word)
        if icf == 0:
            return 0
        return np.log(icf)
        
class Doc(dict):
    def __init__(self):
        self['text'] = dict()
        self['text pos'] = dict()
        self['title pos'] = dict()
        self['title'] = dict()
        self['len'] = 0
        self['id'] = None
    def count_word(self, word):
        in_title = 0
        in_text = 0
        if word in self['title']:
            in_title = self['title'][word]
        if word in self['text']:
            in_text = self['text'][word]
        return in_title, in_text
    def get_pos(self, word):
        in_title = []
        in_text = []
        if word in self['title pos']:
            in_title = self['title pos'][word]
        if word in self['text pos']:
            in_text = self['text pos'][word]
        return in_title, in_text

In [45]:
def extract_info(doc_id):
    with open(DATA_FOLDER + str(doc_id) + '.dat', 'rb') as f:
        cur_doc = dill.load(f)
    doc = Doc()
    #print(cur_doc['title'])
    title = cur_doc['title']
    text = cur_doc['text']
    pos = 0
    if title == None:
        title = ''
    for word in word_tokenize(title):
        if word in doc['title']:
            doc['title'][word] += 1
        else:
            doc['title'][word] = 1
        
        if word in doc['title pos']:
            doc['title pos'][word].append(pos)
        else:
            doc['title pos'][word] = [pos]
        pos += 1
    for word in word_tokenize(text):
        if word in doc['text']:
            doc['text'][word] += 1
        else:
            doc['text'][word] = 1
        
        if word in doc['text pos']:
            doc['text pos'][word].append(pos)
        else:
            doc['text pos'][word] = [pos]
        pos += 1
    doc['id'] = doc_id
    doc['len'] = pos
    with open(DICT_FOLDER + str(doc_id) + '.dat', 'wb') as f:
        dill.dump(doc, f) 

In [46]:
def wrapper_info(doc_id):
    extract_info(doc_id)
    with mutex:
        val.value += 1
        if val.value % 100 == 0:
            clear_output(True)
            print(val.value, flush=True)
    return None

In [47]:
def extract_corpus_stats(dict_id):
    with open(DICT_FOLDER + str(dict_id) + '.dat', 'rb') as f:
        cur_doc = dill.load(f)
    s = set(cur_doc['title'].keys())
    s.update(cur_doc['text'].keys())
    corpus_stats['count docs'] += 1
    for word in s:
        if word in corpus_stats['df']:
            corpus_stats['df'][word] += 1
        else:
            corpus_stats['df'][word] = 1
    for word in cur_doc['text']:
        if word in corpus_stats['cf']:
            corpus_stats['cf'][word] += cur_doc['text'][word]
        else:
            corpus_stats['cf'][word] = cur_doc['text'][word]
    for word in cur_doc['title']:
        if word in corpus_stats['cf']:
            corpus_stats['cf'][word] += cur_doc['title'][word]
        else:
            corpus_stats['cf'][word] = cur_doc['title'][word]
    corpus_stats['count lemms'] += cur_doc['len']        

In [62]:
with open(DICT_FOLDER + 'corpus_stats.dat', 'rb') as f:
    corpus_stats = dill.load(f)

***Вспомогательные функции***

In [32]:
def load_sub(name):
    sub = dict()
    with open(name, 'r') as f:
        submission = list(f)
    for num, line in enumerate(submission):
        if num == 0:
            continue
        line = line.split(',')
        if int(line[0]) in sub:
            sub[int(line[0])].append(int(line[1]))
        else:
            sub[int(line[0])] = [int(line[1])]
    return sub

In [19]:
def load_queue():
    stemmer = Mystem()
    stemmer_eng = Stemmer.Stemmer('english')
    orig_q = dict()
    ext_q = dict()
    with open('queue.txt', 'r') as f1:
        with open('ext.txt', 'r') as f2:
            for i in range(399):
                line = f1.readline().strip()
                line = line.split('\t')
                orig_q[int(line[0])] = line[1]
                line = f2.readline()
                line = line.split('\t')
                ext_q[int(line[0])] = line[1]
            for i in orig_q:
                orig_q[i] = process_0(orig_q[i])
                ext_q[i] = process_0(ext_q[i])
                orig_q[i] = ' '.join(list(filter(lambda x: not(x in stopwords_rus or x in stopwords_eng),nltk.tokenize.word_tokenize(orig_q[i]))))
                ext_q[i] = ' '.join(list(filter(lambda x: not(x in stopwords_rus or x in stopwords_eng),nltk.tokenize.word_tokenize(ext_q[i]))))
                orig_q[i] = ''.join(stemmer.lemmatize(orig_q[i]))
                ext_q[i] = ''.join(stemmer.lemmatize(ext_q[i]))
                orig_q[i] = ' '.join(stemmer_eng.stemWords(orig_q[i].split(' ')))
                ext_q[i] = ' '.join(stemmer_eng.stemWords(ext_q[i].split(' ')))
                orig_q[i] = ' '.join(list(map(process_1, nltk.tokenize.word_tokenize(orig_q[i]))))
                ext_q[i] = ' '.join(list(map(process_1, nltk.tokenize.word_tokenize(ext_q[i]))))
    return orig_q, ext_q

***bm-25***

$$ score(D,Q) = \sum_{i=0}^n IDF(q_i) \frac{f(q_i,D)*(k_1 + 1)}{f(q_i,D) + k_1*(1 - b + b \frac{|D|}{avgdl})} $$  

$$IDF(q_i) = log{\frac{N - n(q_i) + 0.5}{n(q_i) + 0.5}}$$

In [29]:
def single_score(q_i, doc, corpus_statistics):
    in_title, in_text = doc.count_word(q_i)
    tf = (in_title + in_text)/doc['len']
    log_idf = corpus_statistics.log_idf(q_i)
    avgdl = corpus_statistics['avgdl']
    return log_idf * tf*(k1 + 1) / (tf + k1 * (1 - b + b*doc['len']/avgdl))

In [30]:
def bm25(q, doc, corpus_statistics):
    score = 0
    if doc['len'] == 0:
        return 0
    for q_i in nltk.tokenize.word_tokenize(q):
        score += single_score(q_i, doc, corpus_statistics)
    return score

In [31]:
def scoring_bm25(clean_q, clean_e, corpus_statistics, queue_to_nums):
    final_res = dict()
    counter = 0
    for queue_id in range(1, len(clean_q) + 1):
        nums_for_queue = queue_to_nums[queue_id]
        final_res[queue_id] = dict()
        for doc_id in nums_for_queue:
            score = 0
            with open(DICT_FOLDER + str(doc_id) + '.dat', 'rb') as f:
                doc = dill.load(f)
            q_i = clean_q[queue_id]#list(nltk.tokenize.word_tokenize(clean_q[queue_id]))
            e_i = clean_e[queue_id]#list(nltk.tokenize.word_tokenize(clean_e[queue_id]))
            #print(doc['len'], doc_id)
            score = bm25(q_i, doc, corpus_statistics)
            final_res[queue_id][doc_id] = score
        clear_output(True)
        print(queue_id)
    sub = dict()
    for queue_id in range(1, len(clean_q) + 1):
        idx = []
        values = []
        for i in final_res[queue_id]:
            values.append(final_res[queue_id][i])
            idx.append(i)
        values = np.array(values)
        idx = np.array(idx)
        idx = idx[np.argsort(values)[::-1][:10]]
        sub[queue_id] = idx
    return sub 

***bm-25 вторая формула***

$$score(D,Q) = \sum_{i=0}^n IDF(q_i) \frac{f(q_i,D)*(k_1 + 1)}{f(q_i,D) + k_1*(1 - b + b \frac{|D|}{avgdl})}*\frac{(k_2 + 1)*qf_i}{k_2 + qf_i}$$

In [25]:
def single_score_2 (q_i, doc, corpus_statistics):
    in_title, in_text = doc.count_word(q_i)
    tf = (in_title + in_text)/doc['len']
    log_idf = corpus_statistics.log_idf(q_i)
    avgdl = corpus_statistics['avgdl']
    return log_idf * tf*(k1 + 1) / (tf + k1 * (1 - b + b*doc['len']/avgdl))

In [26]:
def bm25_2(q, doc, corpus_statistics):
    score = 0
    if doc['len'] == 0:
        return 0
    freqs = get_freq(q)
    for q_i in nltk.tokenize.word_tokenize(q):
        score += single_score(q_i, doc, corpus_statistics) * (k2 + 1) * freqs[q_i]/(k2 + freqs[q_i])
    return score

In [27]:
def get_freq(q):
    freqs = {}
    for q_i in nltk.word_tokenize(q):
        if q_i in freqs:
            freqs[q_i] += 1
        else:
            freqs[q_i] = 1
    for q_i in q.split(' '):
        freqs[q_i] /= len(list(nltk.word_tokenize(q)))
    return freqs

In [28]:
def scoring_bm25(clean_q, clean_e, corpus_statistics, queue_to_nums):
    final_res = dict()
    counter = 0
    for queue_id in range(1, len(clean_q) + 1):
        nums_for_queue = queue_to_nums[queue_id]
        final_res[queue_id] = dict()
        for doc_id in nums_for_queue:
            score = 0
            with open(DICT_FOLDER + str(doc_id) + '.dat', 'rb') as f:
                doc = dill.load(f)
            q_i = clean_q[queue_id]#list(nltk.tokenize.word_tokenize(clean_q[queue_id]))
            e_i = clean_e[queue_id]#list(nltk.tokenize.word_tokenize(clean_e[queue_id]))
            #print(doc['len'], doc_id)
            score = bm25(q_i, doc, corpus_statistics)
            final_res[queue_id][doc_id] = score
        clear_output(True)
        print(queue_id)
    sub = dict()
    for queue_id in range(1, len(clean_q) + 1):
        idx = []
        values = []
        for i in final_res[queue_id]:
            values.append(final_res[queue_id][i])
            idx.append(i)
        values = np.array(values)
        idx = np.array(idx)
        idx = idx[np.argsort(values)[::-1][:10]]
        sub[queue_id] = idx
    return sub 

***Формула из статьи 'Алгоритм текстового ранжирования Яндекса на РОМИП-2006'***

$$Score = W_{single} + W_{pair} + k_1*W_{AllWords} + k_2*W_{Phrase} + k_3*W_{HalfPhrase} + W_{PRF}$$

$$ W_{single} = log{p}*(TF_1 + s*TF_2), s = 0.5 $$

$$ TF_1 = \frac{TF}{TF + k_1 + k_2*DocLength}, k_1 =1, k_2 = \frac{1}{350}$$

$$ TF_2 = \frac{Hdr}{1 + Hdr} $$
$$ TF_2 = W_{title} $$

***В качестве p я брал ICF и IDF***

$$ p = ICF $$ <br>
$$ p = IDF $$ <br>

***Учет пар слов***

Слова из запроса встречаются в тексте подряд - +1<br>
Слова из запроса встречаются в тексте через одно слова - +0.5<br>
Слова из запроса встречаются в тексте в обратном порядке - +0.5<br>
Слова из запроса, которые идут через слово, встречаются в тексте подряд - 0.1<br> 

$$ W_{pair} = 0.3*(log{p_1} + log{p_2}) * \frac{TF}{1 + TF} $$

$$ p = ICF $$ <br>
$$ p = IDF $$ <br>

***Учет всех слов***

$$W_{AllWords} = 0.2 * \sum log(p_i) * 0.03^{N_{miss}}$$

$$ p = ICF $$ <br>
$$ p = IDF $$ 

$$W_{phrase} = 0.1 * \sum log(p_i)*\frac{TF}{1 + TF} $$

***Учет половины фразы***

$$W_{halfphrase} = 0.02 * \sum log(p_i) *\frac{TF}{1 + TF} $$

In [67]:
def single_word(word, corpus_stats, doc):
    in_title, in_text= doc.count_word(word)
    tf = in_title + in_text
    score1 = tf / (tf + 1.0 + 1.0/1000.0 * doc['len'])
    hdr = in_title * 3.0 + in_text
    score2 = int(in_title > 0)#hdr / (1.0 + hdr)
    return corpus_stats.log_idf(word) * (score1 + 1.0 * score2)

In [75]:
def pair_count(w1, w2, pos1, pos2):
    count_row = 0
    count_inverse = 0
    count_skip = 0
    for p1 in pos1:
        if (p1 + 1) in pos2:
            count_row += 1
        if (p1 - 1) in pos2:
            count_inverse += 1
        if (p1 + 2) in pos2:
            count_skip += 1
    return count_row, count_inverse, count_skip

In [54]:
def count_miss(words, doc, th, corpus_stats):
    count_miss = 0
    for w in words:
        w_idf = corpus_stats.log_idf(w)
        in_title, in_text = doc.count_word(w)
        all_count = in_title + in_text
        if all_count == 0:
            if w_idf > th:
                count_miss += 1
            continue
    return count_miss

In [55]:
def score_words(words, doc, corpus_stats):
    score = 0
    for w in words:
        score += single_word(w, corpus_stats, doc)
    return score

In [56]:
def score_include_all_words(words, doc, th, corpus_stats):
    score = 0
    for w in words:
        score += corpus_stats.log_idf(w)
    score *= 0.03 ** (count_miss(words, doc, th, corpus_stats))
    return score

In [76]:
def score_query_pairs(words, doc, corpus_stats):
    score = 0
    for i in range(len(words) - 1):
        w1 = words[i]
        w2 = words[i + 1]
        in_title1, in_text1 = doc.get_pos(w1)
        pos1 = set(in_title1)
        pos1.update(in_text1)
        in_title2, in_text2 = doc.get_pos(w2)
        pos2 = set(in_title2)
        pos2.update(in_text2)
        count_row, count_inverse, count_skip = pair_count(w1, w2, pos1, pos2)
        pair_tf = count_row * 1.5 + count_inverse * 0.7 + count_skip*0.5
        score += (corpus_stats.log_idf(w1) + corpus_stats.log_idf(w2))*pair_tf/(1 + pair_tf)
    return score

In [58]:
def score_phrases(half_phrase, phrase, doc_num, corpus_stats):
    with open(EXTRACTED + str(doc_num) + '.dat', 'rb') as f:
        cur_doc = dill.load(f)
    tf_title = cur_doc['title'].count(phrase)
    tf_body = cur_doc['text'].count(phrase)
    tf_phrase = tf_body + 2.0 * tf_title
    tf_title = doc['title'].count(half_phrase)
    tf_body = cur_doc['text'].count(half_phrase)
    tf_half_phrase = tf_body + 2.0 * tf_title
    log_idf_hp = 0
    for q_i in nltk.word_tokenize(half_phrase):
        log_idf_hp += corpus_stats.log_idf(q_i)
    log_idf_p = 0
    for q_i in nltk.word_tokenize(phrase):
        log_idf_p += corpus_stats.log_idf(q_i)
    W_p = log_idf_p * (tf_phrase / ( 1 + tf_phrase))
    W_hp = log_idf_hp * (tf_half_phrase / (1 + tf_half_phrase))
    return 0.2 * W_p + 0.01 * W_hp

In [59]:
def scoring_romip(clean_q, clean_e, corpus_stats, queue_to_nums):
    final_res = dict()
    counter = 0
    for queue_id in range(1, len(clean_q) + 1):
        nums_for_queue = queue_to_nums[queue_id]
        final_res[queue_id] = dict()
        for doc_id in nums_for_queue:
            score = 0
            with open(DICT_FOLDER + str(doc_id) + '.dat', 'rb') as f:
                doc = dill.load(f)
            q_i = list(nltk.tokenize.word_tokenize(clean_q[queue_id]))
            e_i = list(nltk.tokenize.word_tokenize(clean_e[queue_id]))
            score += 0.3 * score_words(e_i, doc, corpus_stats)
            score += 0.3 * 0.3 * score_query_pairs(e_i, doc, corpus_stats)
            score += score_words(q_i, doc, corpus_stats)
            score += 0.3 * score_query_pairs(q_i, doc, corpus_stats)
            #score += score_phrases(q_i, '', doc_id, corpus_stats)
            idf_array = []
            for w in q_i:
                idf_array.append(corpus_stats.log_idf(w))
            med_idf = np.median(idf_array)/2
            score += 0.2 * score_include_all_words(q_i, doc, med_idf, corpus_stats)
            final_res[queue_id][doc_id] = score
        clear_output(True)
        print(queue_id)
    sub = dict()
    vals = dict()
    for queue_id in range(1, len(clean_q) + 1):
        idx = []
        values = []
        for i in final_res[queue_id]:
            idx.append(i)
            values.append(final_res[queue_id][i])
        values = np.array(values)
        idx = np.array(idx)
        idx = idx[np.argsort(values)[::-1][:15]]
        vals[queue_id] = values[np.argsort(values)[::-1][:15]]
        sub[queue_id] = idx
    return sub, vals

In [23]:
clean_q, clean_e = load_queue()

In [64]:
num_for_queue = dict()
with open('sample.technosphere.ir1.textrelevance.submission.txt', 'r') as f:
    lines = list(f)
for line in lines:
    line = line.strip()
    line = line.split(',')
    if line[0] == 'QueryId':
        continue
    q_id = int(line[0])
    doc_id = int(line[1])
    if q_id in num_for_queue:
        num_for_queue[q_id].append(doc_id)
    else:
        num_for_queue[q_id] = [doc_id]

In [69]:
sub, vals = scoring_romip(clean_q, clean_e, corpus_stats, num_for_queue)

399


***Embeddings***

Universal Sentence Encoder Multilingual

In [87]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

Instructions for updating:
non-resource variables are not supported in the long term


In [88]:
module_url = 'https://tfhub.dev/google/universal-sentence-encoder-multilingual/1'  #@param ['https://tfhub.dev/google/universal-sentence-encoder-multilingual/1', 'https://tfhub.dev/google/universal-sentence-encoder-multilingual-large/1', 'https://tfhub.dev/google/universal-sentence-encoder-xling-many/1']

# Set up graph.
g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  multiling_embed = hub.Module(module_url)
  embedded_text = multiling_embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

# Initialize session.
session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [91]:
res_sub = dict()
res_vals = dict()
for j in sub:
    cur_top = np.array(sub[j])
    cur_queue = clean_q[j]
    corpus = [cur_queue]
    for doc_id in cur_top:
        with open(EXTRACTED_RAW + str(doc_id) + '.dat', 'r') as f:
            doc = f.read()
        if len(doc) > 700_000:
            doc = doc[:700_000]
        corpus.append(doc)
    embeddings_res = session.run(embedded_text, feed_dict={text_input : corpus})
    cosine_dists = sklearn.metrics.pairwise.cosine_similarity(embeddings_res)[0,1:]
    res_vals[j] = cosine_dists
    with open('checkpoints/' + str(j) + '.dat', 'wb') as f:
        dill.dump(sub, f)
    clear_output(True)
    print(j)

399


В качестве финального ранжирования:<br>
Используя алгоритм РОМИП получаем топ 15 текстов и score для каждого из них. Для этих текстов вычисляем эмбединги. Используя их, вычисляем cosine_similarity. Получаем два вектора $$score_{bm25}$$ и $$score_{nn}$$.
Окончательно ранжируем - $$ score_{final} = 0.8 \frac{score_{bm25}}{|score_{bm25}|} + 0.2\frac{score_{nn}}{|score_{nn}|}$$

In [92]:
with open('res_vals.dump', 'wb') as f:
    dill.dump(res_vals, f)

In [83]:
with open('res_vals.dump', 'rb') as f:
    res_vals = dill.load(f)

In [93]:
final_sub = dict()
for s in sub:
    doc_ids = np.array(sub[s])
    nn_weights = np.array(res_vals[s])
    bm_weights = np.array(vals[s])
    #nn_weights = np.abs(nn_weights)
    nn_weights /= sum(nn_weights)
    bm_weights /= sum(bm_weights)
    final_weights = 0.8 * bm_weights + 0.2 * nn_weights
    final_sub[s] = doc_ids[np.argsort(final_weights)[::-1]][:10]

In [94]:
def write_sub(name, sub):
    with open(name + '.sub', 'w') as f:
        f.write('QueryId,DocumentId' + '\n')
        for q_id in sub:
            for res in sub[q_id]:
                f.write(str(q_id) + ',' + str(res) + '\n')

In [95]:
write_sub('final_sub', final_sub)

***Финальное решение***

In [ ]:
mutex = Lock()
val = Value('i', 0)
with Pool(processes=6) as pool1:
    pool1.map(wrapper_extract_raw, [i for i in range(1, DOC_NUM + 1)])

In [ ]:
mutex = Lock()
val = Value('i', 0)
with Pool(processes=6) as pool1:
    pool1.map(wrapper_extract, [i for i in range(1, DOC_NUM + 1)])

In [77]:
for i in range(1, DOC_NUM + 1):
    extract_corpus_stats(i)
    #clear_output(True)
    if i % 100 == 0:
        clear_output(True)
        print(i)

38100


In [78]:
corpus_stats['avgdl'] = corpus_stats['count lemms'] / corpus_stats['count docs']

In [79]:
clean_q, clean_e = load_queue()

In [80]:
num_for_queue = dict()
with open('sample.technosphere.ir1.textrelevance.submission.txt', 'r') as f:
    lines = list(f)
for line in lines:
    line = line.strip()
    line = line.split(',')
    if line[0] == 'QueryId':
        continue
    q_id = int(line[0])
    doc_id = int(line[1])
    if q_id in num_for_queue:
        num_for_queue[q_id].append(doc_id)
    else:
        num_for_queue[q_id] = [doc_id]

In [81]:
sub, vals = scoring_romip(clean_q, clean_e, corpus_stats, num_for_queue)

399


In [ ]:
res_sub = dict()
res_vals = dict()
for j in sub:
    cur_top = np.array(sub[j])
    cur_queue = clean_q[j]
    corpus = [cur_queue]
    for doc_id in cur_top:
        with open(EXTRACTED_RAW + str(doc_id) + '.dat', 'r') as f:
            doc = f.read()
        if len(doc) > 700_000:
            doc = doc[:700_000]
        corpus.append(doc)
    embeddings_res = session.run(embedded_text, feed_dict={text_input : corpus})
    cosine_dists = sklearn.metrics.pairwise.cosine_similarity(embeddings_res)[0,1:]
    res_vals[j] = cosine_dists
    with open('checkpoints/' + str(j) + '.dat', 'wb') as f:
        dill.dump(sub, f)
    clear_output(True)
    print(j)

In [ ]:
final_sub = dict()
for s in sub:
    doc_ids = np.array(sub[s])
    nn_weights = np.array(res_vals[s])
    bm_weights = np.array(vals[s])
    #nn_weights = np.abs(nn_weights)
    nn_weights /= sum(nn_weights)
    bm_weights /= sum(bm_weights)
    final_weights = 0.8 * bm_weights + 0.2 * nn_weights
    final_sub[s] = doc_ids[np.argsort(final_weights)[::-1]][:10]

In [ ]:
write_sub('final_sub', final_sub)